In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
import os
import json
import ast

# === Load và xử lý dữ liệu ===
df = pd.read_csv("results.csv")
df

,url,title,publish_date,summary,event_type,trigger,arg_subject,arg_time,arg_location,tokens,bio_tags,argument_tags
0,https://baochinhphu.vn/ban-hanh-vi-tri-viec-la...,Ban hành vị trí việc làm của đại biểu Quốc hội...,06/09/2024 16:44,Ủy ban Thường vụ Quốc hội có Nghị quyết số 115...,Legislation,ban hành,Ủy ban Thường vụ Quốc hội,NaN,NaN,"['Ủy', 'ban_Thường_vụ_Quốc_hội', 'có', 'Nghị_q...","['O', 'B-Legislation', 'O', 'O', 'O', 'O', 'O'...","['O', 'B-Arg-Subject', 'O', 'O', 'O', 'O', 'O'..."
1,https://baochinhphu.vn/thiet-ke-co-che-chinh-s...,"Thiết kế cơ chế, chính sách bảo vệ môi trường ...",06/09/2024 16:40,"Sáng 6/9, Phó Thủ tướng Trần Hồng Hà chủ trì c...",Legislation,"chủ trì, cuộc họp, cho ý kiến, hoàn thiện, Dự ...",Phó Thủ tướng Trần Hồng Hà,Sáng 6/9,NaN,"['Sáng', '6/9', ',', 'Phó', 'Thủ_tướng', 'Trần...","['O', 'O', 'O', 'O', 'O', 'O', 'B-Legislation'...","['B-Arg-Time', 'I-Arg-Time', 'O', 'B-Arg-Subje..."
2,https://baochinhphu.vn/bo-truong-bo-giao-thong...,Bộ trưởng Bộ Giao thông vận tải thôi kiêm giữ ...,06/09/2024 16:12,Chủ tịch Quốc hội Trần Thanh Mẫn đã ký ban hàn...,Legislation,"ký, ban hành, Nghị quyết",Chủ tịch Quốc hội Trần Thanh Mẫn,NaN,NaN,"['Chủ_tịch', 'Quốc_hội', 'Trần_Thanh_Mẫn', 'đã...","['O', 'O', 'O', 'O', 'B-Legislation', 'B-Legis...","['B-Arg-Subject', 'B-Arg-Subject', 'B-Arg-Subj..."
3,https://baochinhphu.vn/bo-chinh-tri-ban-bi-thu...,"Bộ Chính trị, Ban Bí thư xem xét, thi hành kỷ ...",06/09/2024 16:05,"Ngày 6/9, tại Trụ sở Trung ương Đảng, Bộ Chính...",Inspection,"xem xét, thi hành, kỷ luật","Bộ Chính trị, Ban Bí thư",Ngày 6/9,Trụ sở Trung ương Đảng,"['Ngày', '6/9', ',', 'tại', 'Trụ_sở', 'Trung_ư...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","['B-Arg-Time', 'I-Arg-Time', 'O', 'O', 'B-Arg-..."
4,https://baochinhphu.vn/quang-nam-quyet-tam-hoa...,"Quảng Nam quyết tâm hoàn thành việc đăng ký, c...",06/09/2024 15:01,Sở NN&PTNT Quảng Nam đã thành lập tổ công tác ...,Cooperation,"thành lập tổ công tác phối hợp, hỗ trợ, hướng dẫn",Sở NN&PTNT Quảng Nam,30/9,NaN,"['Sở', 'NN&PTNT', 'Quảng_Nam', 'đã', 'thành_lậ...","['O', 'O', 'O', 'O', 'B-Cooperation', 'B-Coope...","['O', 'B-Arg-Subject', 'B-Arg-Subject', 'O', '..."
...,...,...,...,...,...,...,...,...,...,...,...,...
5809,https://baochinhphu.vn/chu-tich-nuoc-du-chuong...,Chủ tịch nước dự Chương trình 'Xuân Biên phòng...,20/01/2024 23:23,"Tối 20/1, Chương trình “Xuân Biên phòng ấm lòn...",Celebration,"Chương trình, phối hợp tổ chức, diễn ra","Bộ Tư lệnh Bộ đội Biên phòng, UBND tỉnh Kiên G...",Tối 20/1,"TP. Hà Tiên, tỉnh Kiên Giang","['Tối', '20/1', ',', 'Chương_trình', '“', 'Xuâ...","['O', 'O', 'O', 'B-Celebration', 'O', 'O', 'O'...","['B-Arg-Time', 'I-Arg-Time', 'O', 'O', 'O', 'B..."
5810,https://baochinhphu.vn/thu-tuong-pham-minh-chi...,Thủ tướng Phạm Minh Chính tới thủ đô Bucharest...,20/01/2024 21:42,Sau khi kết thúc tốt đẹp chuyến thăm chính thứ...,Visiting,"tới, bắt đầu thăm chính thức",Thủ tướng Chính phủ Phạm Minh Chính và Phu nhâ...,chiều 20/1,"Thủ đô Bucharest, Romania","['Sau', 'khi', 'kết_thúc', 'tốt_đẹp', 'chuyến'...","['O', 'O', 'O', 'O', 'O', 'B-Visiting', 'B-Vis...","['O', 'O', 'O', 'O', 'O', 'O', 'B-Arg-Subject'..."
5811,https://baochinhphu.vn/thu-tuong-tham-trung-ta...,Thủ tướng thăm Trung tâm thương mại lớn nhất c...,20/01/2024 19:53,"Sáng 20/1, Thủ tướng Chính phủ Phạm Minh Chính...",Visiting,thăm,Thủ tướng Chính phủ Phạm Minh Chính,Sáng 20/1,Trung tâm Thương mại Thăng Long ở Thủ đô Budap...,"['Sáng', '20/1', ',', 'Thủ_tướng', 'Chính_phủ'...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Visitin...","['B-Arg-Time', 'I-Arg-Time', 'O', 'B-Arg-Subje..."
5812,https://baochinhphu.vn/viet-nam-la-mot-tam-guo...,Việt Nam là một tấm gương về con đường phát tr...,20/01/2024 19:02,"Sáng 20/1, tại Budapest, trong chương trình ch...",Diplomatic Reception,tiếp,"Thủ tướng Phạm Minh Chính, Chủ tịch Đảng Công ...",Sáng 20/1,Budapest,"['Sáng', '20/1', ',', 'tại', 'Budapest', ',', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","['B-Arg-Time', 'I

In [ ]:
df['trigger'][1]

'Phiên chất vấn và trả lời chất vấn'

In [ ]:
df['tokens'][1]

"['Phiên', 'chất_vấn', 'và', 'trả_lời', 'chất_vấn', 'tại', 'kỳ', 'họp', 'thứ', '9', 'Quốc_hội', 'khóa', 'XV', 'được', 'bắt_đầu', 'từ', 'sáng_mai', '19/6', 'và', 'diễn', 'ra', 'trong', '1,5', 'ngày', '(', '19-20', '/', '6', ')', 'đối_với', '2', 'nhóm', 'vấn_đề', '.', 'Chủ_tịch', 'Quốc_hội', 'sẽ', 'có', 'phát_biểu', 'khai_mạc', ',', 'kết_thúc', 'và', 'điều_hành', 'các', 'nội_dung', 'của', 'phiên', 'Phiên', 'chất_vấn', '.']"

In [ ]:
df['bio_tags'][1]

"['B-Meeting', 'B-Meeting', 'B-Meeting', 'B-Meeting', 'B-Meeting', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Meeting', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Meeting', 'O']"

In [ ]:
# Định nghĩa danh sách
bio_tags_list = ['Sáng', '6/9', ',', 'Phó', 'Thủ_tướng', 'Trần_Hồng_Hà', 'chủ_trì', 'cuộc', 'họp', ',', 'cho', 'ý_kiến', 'hoàn_thiện', 'Dự_thảo', 'nghị_định', 'sửa_đổi', 'bổ_sung', 'Nghị_định', 'số', '08/2022', '/', 'NĐ-CP', 'quy_định', 'chi_tiết', 'một_số', 'điều', 'của', 'Luật_Bảo_vệ', 'môi_trường', '(', 'Dự_thảo', 'Nghị_định', ')', '.']

# Đếm số lượng phần tử trong danh sách
count = len(bio_tags_list)

# In ra số lượng phần tử
print(f"Số lượng phần tử có trong danh sách là: {count}")


Số lượng phần tử có trong danh sách là: 34


# ***Pipeline Approach***

***Phân biệt trigger tagging và argument detection***

| Tiêu chí                       | **Trigger Tagging**                                   | **Argument Detection**                                    |
| ------------------------------ | ----------------------------------------------------- | --------------------------------------------------------- |
| **Mục tiêu**                   | Nhận diện các từ/nhóm từ **gợi ra sự kiện** (trigger) | Nhận diện các **thành phần** liên quan đến sự kiện (role) |
| **Ví dụ nhãn**                 | `B-Trigger`, `I-Trigger`, `O`                         | `B-Agent`, `I-Agent`, `B-Location`, `O`, ...              |
| **Ý nghĩa nhãn**               | "Đây là trung tâm của sự kiện"                        | "Đây là ai, ở đâu, khi nào, với ai..."                    |
| **Số lượng trigger/arg**       | Thường chỉ có 1 trigger / câu                         | Có thể có **nhiều** argument (Agent, Location, Time, ...) |
| **Gắn với event\_type không?** | Có – mỗi trigger phải gắn với 1 loại sự kiện cụ thể   | Có – nhưng argument chỉ có ý nghĩa khi gắn với trigger đó |
| **Đầu vào khi huấn luyện?**    | Văn bản thường + nhãn trigger                         | Văn bản + thông tin trigger đã biết (trong pipeline)      |

***Trigger Tagging***

| Token    | Label         |
| -------- | ------------- |
| Thủ      | O             |
| tướng    | O             |
| Nguyễn   | O             |
| Xuân     | O             |
| Phúc     | O             |
| **chủ**  | **B-Trigger** |
| **trì**  | **I-Trigger** |
| hội nghị | O             |
| cấp cao  | O             |
| ASEAN    | O             |
| tại      | O             |
| Hà Nội   | O             |


***Argument Detection***

| Token    | Label      |
| -------- | ---------- |
| Thủ      | B-Subject  |
| tướng    | I-Subject  |
| Nguyễn   | I-Subject  |
| Xuân     | I-Subject  |
| Phúc     | I-Subject  |
| chủ      | O          |
| trì      | O          |
| hội nghị | O          |
| cấp cao  | O          |
| ASEAN    | O          |
| tại      | O          |
| Hà Nội   | B-Location |


## ***Trigger Tagging***

In [ ]:
import pandas as pd
import ast
from sklearn.model_selection import StratifiedShuffleSplit
import os
import json

# === Load dữ liệu từ file mới ===
df = pd.read_csv("results.csv")
df["tokens"] = df["tokens"].apply(ast.literal_eval)
df["bio_tags"] = df["bio_tags"].apply(ast.literal_eval)
df = df[df["tokens"].str.len() == df["bio_tags"].str.len()]

# === Gán nhãn chính dùng để phân tầng ===
def get_primary_label(tags):
    for tag in tags:
        if tag != "O":
            return tag
    return "O"

df["main_label"] = df["bio_tags"].apply(get_primary_label)

# Bỏ các nhãn có số lượng quá ít (ít nhất 3 để chia train/dev/test hợp lý)
label_counts = df["main_label"].value_counts()
labels_to_keep = label_counts[label_counts >= 3].index
df = df[df["main_label"].isin(labels_to_keep)]

# === Chia tập train (80%) và temp (20%) ===
splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_idx, temp_idx in splitter.split(df, df["main_label"]):
    train_df = df.iloc[train_idx]
    temp_df = df.iloc[temp_idx]

# Lọc temp để giữ lại nhãn có ít nhất 2 mẫu (cho dev/test)
temp_label_counts = temp_df["main_label"].value_counts()
temp_labels_to_keep = temp_label_counts[temp_label_counts >= 2].index
temp_df = temp_df[temp_df["main_label"].isin(temp_labels_to_keep)]

# === Chia tiếp dev/test từ temp (50%/50%) ===
splitter2 = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=42)
for dev_idx, test_idx in splitter2.split(temp_df, temp_df["main_label"]):
    dev_df = temp_df.iloc[dev_idx]
    test_df = temp_df.iloc[test_idx]

# === Chuyển sang JSONL format ===
def convert_to_jsonl_format(df):
    return [{"tokens": row["tokens"], "labels": row["bio_tags"]} for _, row in df.iterrows()]

train_data = convert_to_jsonl_format(train_df)
dev_data = convert_to_jsonl_format(dev_df)
test_data = convert_to_jsonl_format(test_df)

output_dir = "/content/drive/MyDrive/DS200_project/Vietnamese_Political_Events_Extraction/trigger_tagging_data"
os.makedirs(output_dir, exist_ok=True)

def save_jsonl(data, filename):
    with open(filename, "w", encoding="utf-8") as f:
        for item in data:
            json.dump(item, f, ensure_ascii=False)
            f.write("\n")

save_jsonl(train_data, f"{output_dir}/train.json")
save_jsonl(dev_data, f"{output_dir}/dev.json")
save_jsonl(test_data, f"{output_dir}/test.json")

In [ ]:
pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=f6d08777e6a526e0688469142baf3cf459fa53ecdc063b3e123a1129e803ad75
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval


In [ ]:
from datasets import Dataset
from transformers import (
    AutoTokenizer, AutoModelForTokenClassification, TrainingArguments,
    Trainer, DataCollatorForTokenClassification, EarlyStoppingCallback
)
from seqeval.metrics import classification_report, precision_score, recall_score, f1_score
import numpy as np
import json
import warnings
warnings.filterwarnings('ignore')

# === Load JSONL data ===
def load_jsonl(filename):
    with open(filename, "r", encoding="utf-8") as f:
        return [json.loads(line) for line in f]

train_data = load_jsonl(f"{output_dir}/train.json")
dev_data = load_jsonl(f"{output_dir}/dev.json")
test_data = load_jsonl(f"{output_dir}/test.json")

# === Tokenizer và labels ===
model_checkpoint = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

all_labels = set()
for item in train_data + dev_data + test_data:
    all_labels.update(item["labels"])

label_list = sorted(list(all_labels))
label_to_id = {l: i for i, l in enumerate(label_list)}
id_to_label = {i: l for l, i in label_to_id.items()}

def tokenize_and_align_labels(example):
    tokenized = tokenizer(example["tokens"], is_split_into_words=True, truncation=True)
    word_ids = tokenized.word_ids()
    label_ids = []
    previous_word_idx = None

    for word_idx in word_ids:
        if word_idx is None:
            label_ids.append(-100)
        elif word_idx != previous_word_idx:
            label_ids.append(label_to_id[example["labels"][word_idx]])
        else:
            label_ids.append(label_to_id[example["labels"][word_idx]] if example["labels"][word_idx].startswith("I") else -100)
        previous_word_idx = word_idx

    tokenized["labels"] = label_ids
    return tokenized

train_dataset = Dataset.from_list(train_data).map(tokenize_and_align_labels, batched=False)
dev_dataset = Dataset.from_list(dev_data).map(tokenize_and_align_labels, batched=False)
test_dataset = Dataset.from_list(test_data).map(tokenize_and_align_labels, batched=False)

# === Huấn luyện ===
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/DS200_project/Vietnamese_Political_Events_Extraction/xlmr_trigger_tagging",
    eval_strategy="steps",
    eval_steps=100,
    save_steps=100,
    logging_steps=100,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.01,
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro"
)

def compute_metrics(p):
    predictions = np.argmax(p.predictions, axis=-1)
    labels = p.label_ids
    true_predictions = [
        [id_to_label[p] for (p, l) in zip(pred, lab) if l != -100]
        for pred, lab in zip(predictions, labels)
    ]
    true_labels = [
        [id_to_label[l] for (p, l) in zip(pred, lab) if l != -100]
        for pred, lab in zip(predictions, labels)
    ]
    report = classification_report(true_labels, true_predictions, output_dict=True)
    return {
        "precision": precision_score(true_labels, true_predictions),
        "recall": recall_score(true_labels, true_predictions),
        "f1": f1_score(true_labels, true_predictions),
        "f1_macro": report["macro avg"]["f1-score"],
        "precision_macro": report["macro avg"]["precision"],
        "recall_macro": report["macro avg"]["recall"]
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorForTokenClassification(tokenizer),
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

trainer.train()


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Map:   0%|          | 0/4168 [00:00<?, ? examples/s]

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: quihung (quihung-uit) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss,Precision,Recall,F1,F1 Macro,Precision Macro,Recall Macro
100,0.693800,0.424763,0.490637,0.066599,0.117278,0.030629,0.035045,0.027201
200,0.386400,0.328624,0.506479,0.238434,0.324231,0.181923,0.431440,0.156271
300,0.314400,0.280802,0.562541,0.441281,0.494587,0.348174,0.505104,0.344622
400,0.275100,0.250585,0.606742,0.466701,0.527586,0.439877,0.695913,0.392345
500,0.251200,0.231096,0.616206,0.498729,0.551278,0.466813,0.692840,0.432261
600,0.224000,0.225598,0.601558,0.549568,0.574389,0.525092,0.643917,0.497050
700,0.212800,0.209281,0.658475,0.531266,0.588070,0.529789,0.669299,0.484900
800,0.213000,0.209937,0.653504,0.597356,0.624170,0.596667,0.662492,0.567904
900,0.192100,0.217078,0.700214,0.498729,0.582542,0.537183,0.732691,0.453410
1000,0.185500,0.193576,0.648942,0.577021,0.610872,0.584530,0.655077,0.560991


TrainOutput(global_step=1100, training_loss=0.28436784917658026, metrics={'train_runtime': 281.6635, 'train_samples_per_second': 147.978, 'train_steps_per_second': 18.497, 'total_flos': 784031775170016.0, 'train_loss': 0.28436784917658026, 'epoch': 2.111324376199616})

In [ ]:
# === Đánh giá tập test ===
test_results = trainer.predict(test_dataset)
metrics = compute_metrics(test_results)

print("\n========== KẾT QUẢ TRÊN TẬP TEST ==========")
print(f"Precision (macro): {metrics['precision_macro']:.4f}")
print(f"Recall (macro):    {metrics['recall_macro']:.4f}")
print(f"F1-score (macro):  {metrics['f1_macro']:.4f}")



========== KẾT QUẢ TRÊN TẬP TEST ==========
Precision (macro): 0.6179
Recall (macro):    0.5668
F1-score (macro):  0.5849


In [ ]:
# === Hàm dự đoán trigger từ câu văn ===
import torch

def predict_trigger(sentence, tokenizer, model, id_to_label):
    tokens = sentence.strip().split()
    model.eval()
    inputs = tokenizer(tokens, is_split_into_words=True, return_tensors="pt", truncation=True)
    word_ids = inputs.word_ids()
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1).squeeze().cpu().numpy()

    word_predictions = []
    previous_word_idx = None
    for idx, word_idx in enumerate(word_ids):
        if word_idx is None:
            continue
        if word_idx != previous_word_idx:
            word_predictions.append((tokens[word_idx], id_to_label[predictions[idx]]))
        previous_word_idx = word_idx

    triggers = [token for token, label in word_predictions if label != "O"]
    return triggers, word_predictions

# Ví dụ chạy thử
input_sentence = "Tiếp tục chương trình Kỳ họp thứ 9, chiều 20/6, Quốc hội họp phiên toàn thể tại Hội trường, nghe báo cáo về dự thảo Nghị quyết của Quốc hội về cơ chế xử lý khó khăn, vướng mắc do quy định của pháp luật."
triggers, word_preds = predict_trigger(input_sentence, tokenizer, model, id_to_label)

print("Câu:", input_sentence)
print("Trigger tokens:", triggers)
print("Chi tiết từng từ:", word_preds)


Câu: Tiếp tục chương trình Kỳ họp thứ 9, chiều 20/6, Quốc hội họp phiên toàn thể tại Hội trường, nghe báo cáo về dự thảo Nghị quyết của Quốc hội về cơ chế xử lý khó khăn, vướng mắc do quy định của pháp luật.
Trigger tokens: ['họp', 'phiên', 'nghe', 'báo', 'cáo', 'dự', 'thảo', 'Nghị', 'quyết']
Chi tiết từng từ: [('Tiếp', 'O'), ('tục', 'O'), ('chương', 'O'), ('trình', 'O'), ('Kỳ', 'O'), ('họp', 'O'), ('thứ', 'O'), ('9,', 'O'), ('chiều', 'O'), ('20/6,', 'O'), ('Quốc', 'O'), ('hội', 'O'), ('họp', 'B-Legislation'), ('phiên', 'B-Legislation'), ('toàn', 'O'), ('thể', 'O'), ('tại', 'O'), ('Hội', 'O'), ('trường,', 'O'), ('nghe', 'B-Legislation'), ('báo', 'B-Legislation'), ('cáo', 'B-Legislation'), ('về', 'O'), ('dự', 'B-Legislation'), ('thảo', 'B-Legislation'), ('Nghị', 'B-Legislation'), ('quyết', 'B-Legislation'), ('của', 'O'), ('Quốc', 'O'), ('hội', 'O'), ('về', 'O'), ('cơ', 'O'), ('chế', 'O'), ('xử', 'O'), ('lý', 'O'), ('khó', 'O'), ('khăn,', 'O'), ('vướng', 'O'), ('mắc', 'O'), ('do', 'O'), 

## ***Argument Detection***

In [ ]:
import pandas as pd
import ast
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import XLMRobertaTokenizerFast

# === Load dữ liệu ===
df = pd.read_csv("results.csv")
df["tokens"] = df["tokens"].apply(ast.literal_eval)
df["argument_tags"] = df["argument_tags"].apply(ast.literal_eval)

# Lọc dữ liệu không hợp lệ
df = df[df["tokens"].str.len() == df["argument_tags"].str.len()]
df = df.reset_index(drop=True)

# === Chia train/dev/test ngẫu nhiên ===
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
dev_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# === Chuẩn hóa dữ liệu cho từng tập ===
train_data = [{"tokens": t, "labels": l} for t, l in zip(train_df["tokens"], train_df["argument_tags"])]
dev_data = [{"tokens": t, "labels": l} for t, l in zip(dev_df["tokens"], dev_df["argument_tags"])]
test_data = [{"tokens": t, "labels": l} for t, l in zip(test_df["tokens"], test_df["argument_tags"])]

# === Tokenizer và ánh xạ tag ===
tokenizer = XLMRobertaTokenizerFast.from_pretrained("xlm-roberta-base")
unique_tags = sorted({tag for sample in train_data + dev_data + test_data for tag in sample["labels"]})
tag2id = {tag: i for i, tag in enumerate(unique_tags)}
id2tag = {i: tag for tag, i in tag2id.items()}


In [ ]:
# === Hàm encode token + nhãn ===
def encode_tags(example):
    tokenized = tokenizer(example["tokens"], is_split_into_words=True, truncation=True)
    word_ids = tokenized.word_ids()
    labels = []
    previous_word_idx = None
    for word_idx in word_ids:
        if word_idx is None:
            labels.append(-100)
        elif word_idx != previous_word_idx:
            labels.append(tag2id[example["labels"][word_idx]])
        else:
            labels.append(tag2id[example["labels"][word_idx]])
        previous_word_idx = word_idx
    tokenized["labels"] = labels
    return tokenized

# === Chuyển sang Dataset và mã hóa ===
train_ds = Dataset.from_list(train_data).map(encode_tags)
dev_ds = Dataset.from_list(dev_data).map(encode_tags)
test_ds = Dataset.from_list(test_data).map(encode_tags)


Map:   0%|          | 0/4168 [00:00<?, ? examples/s]

Map:   0%|          | 0/521 [00:00<?, ? examples/s]

Map:   0%|          | 0/522 [00:00<?, ? examples/s]

In [ ]:
import numpy as np
import warnings
from transformers import (
    XLMRobertaForTokenClassification, Trainer, TrainingArguments,
    DataCollatorForTokenClassification
)
from seqeval.metrics import classification_report, f1_score, precision_score, recall_score

warnings.filterwarnings('ignore')

# === Mô hình ===
model = XLMRobertaForTokenClassification.from_pretrained(
    "xlm-roberta-base", num_labels=len(tag2id)
)

# === Tham số huấn luyện ===
args = TrainingArguments(
    output_dir="/content/drive/MyDrive/DS200_project/Vietnamese_Political_Events_Extraction/xlmr_argument_detection",
    eval_strategy="steps",
    eval_steps=100,
    save_steps=100,
    logging_steps=100,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.01,
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
)

# === Hàm tính metrics ===
def compute_metrics(eval_pred):
    predictions = eval_pred.predictions
    labels = eval_pred.label_ids

    predictions = np.argmax(predictions, axis=2)

    true_labels = [
        [id2tag[label] for label, pred in zip(lseq, pseq) if label != -100]
        for lseq, pseq in zip(labels, predictions)
    ]
    true_preds = [
        [id2tag[pred] for label, pred in zip(lseq, pseq) if label != -100]
        for lseq, pseq in zip(labels, predictions)
    ]

    report = classification_report(true_labels, true_preds, output_dict=True)

    return {
        "precision": precision_score(true_labels, true_preds),
        "recall": recall_score(true_labels, true_preds),
        "f1": f1_score(true_labels, true_preds),
        "f1_macro": report["macro avg"]["f1-score"],
        "precision_macro": report["macro avg"]["precision"],
        "recall_macro": report["macro avg"]["recall"],
    }


# === Trainer ===
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=dev_ds,
    tokenizer=tokenizer,
    data_collator=DataCollatorForTokenClassification(tokenizer),
    compute_metrics=compute_metrics,
)

# === Train ===
trainer.train()


Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss,Precision,Recall,F1,F1 Macro,Precision Macro,Recall Macro
100,0.599000,0.410915,0.737498,0.638522,0.684451,0.640932,0.692353,0.635794
200,0.350700,0.340811,0.825604,0.682928,0.747519,0.689992,0.750913,0.643651
300,0.306500,0.307755,0.836177,0.761811,0.797263,0.729112,0.771805,0.695756
400,0.292200,0.306735,0.831678,0.774639,0.802146,0.731758,0.788343,0.685407
500,0.294200,0.290300,0.788366,0.822499,0.805071,0.743294,0.741720,0.748375
600,0.258500,0.287448,0.861124,0.779388,0.818220,0.743367,0.816744,0.685998
700,0.232200,0.295575,0.863583,0.766806,0.812322,0.741417,0.792036,0.696931
800,0.238400,0.259135,0.862349,0.798261,0.829068,0.768936,0.822088,0.724673
900,0.239900,0.265981,0.848424,0.811953,0.829788,0.764967,0.830011,0.715860
1000,0.229800,0.269569,0.848215,0.800296,0.823559,0.764693,0.792359,0.744628


TrainOutput(global_step=5210, training_loss=0.151413712265853, metrics={'train_runtime': 1145.0966, 'train_samples_per_second': 36.399, 'train_steps_per_second': 4.55, 'total_flos': 3702309624405216.0, 'train_loss': 0.151413712265853, 'epoch': 10.0})

In [ ]:
test_preds = trainer.predict(test_ds)
metrics = compute_metrics(test_preds)

print("\n========== ĐÁNH GIÁ TRÊN TẬP TEST ==========")
print(f"Precision (macro): {metrics['precision_macro']:.4f}")
print(f"Recall (macro):    {metrics['recall_macro']:.4f}")
print(f"F1-score (macro):  {metrics['f1_macro']:.4f}")


========== ĐÁNH GIÁ TRÊN TẬP TEST ==========
Precision (macro): 0.8307
Recall (macro):    0.8201
F1-score (macro):  0.8253


In [ ]:
import torch

def predict_argument_tags(sentence: str):
    tokens = sentence.strip().split()
    inputs = tokenizer(tokens, is_split_into_words=True, return_tensors="pt", truncation=True)
    word_ids = inputs.word_ids()

    model.eval()
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=2).squeeze().cpu().numpy()

    predicted_tags = []
    previous_word_idx = None
    for pred, word_idx in zip(predictions, word_ids):
        if word_idx is None:
            continue
        if word_idx != previous_word_idx:
            predicted_tags.append(id2tag[pred])
            previous_word_idx = word_idx

    return list(zip(tokens, predicted_tags))

input_sentence = "Phó Thủ tướng Nguyễn Chí Dũng chủ trì cuộc họp về triển khai nghị quyết Quốc hội."
prediction = predict_argument_tags(input_sentence)

print("Dự đoán Argument Tags:")
print(f"{'Từ':<20} | {'Nhãn'}")
print("-" * 35)
for token, tag in prediction:
    print(f"{token:<20} | {tag}")

Dự đoán Argument Tags:
Từ                   | Nhãn
-----------------------------------
Phó                  | B-Arg-Subject
Thủ                  | B-Arg-Subject
tướng                | B-Arg-Subject
Nguyễn               | B-Arg-Subject
Chí                  | B-Arg-Subject
Dũng                 | B-Arg-Subject
chủ                  | O
trì                  | O
cuộc                 | O
họp                  | O
về                   | O
triển                | O
khai                 | O
nghị                 | O
quyết                | O
Quốc                 | O
hội.                 | O


## ***Event type classification***

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import json
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score
import numpy as np

# === Load và lọc dữ liệu ===
df = pd.read_csv("results.csv")
df = df[df["event_type"].notna() & df["summary"].notna()].reset_index(drop=True)

# === Chia train/dev/test ngẫu nhiên không stratified ===
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
dev_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)


In [ ]:
# === Tokenizer và label mapping ===
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
label_list = sorted(df["event_type"].unique())
label_to_id = {l: i for i, l in enumerate(label_list)}
id_to_label = {i: l for l, i in label_to_id.items()}

# === Chuẩn bị dữ liệu ===
def convert_data(df, label_to_id):
    return [{"text": row["summary"], "label": label_to_id[row["event_type"]]} for _, row in df.iterrows()]

train_data = convert_data(train_df, label_to_id)
dev_data = convert_data(dev_df, label_to_id)

def tokenize(example):
    tokenized = tokenizer(example["text"], truncation=True, padding="max_length", max_length=256)
    tokenized["label"] = example["label"]
    return tokenized

train_dataset = Dataset.from_list(train_data).map(tokenize)
dev_dataset = Dataset.from_list(dev_data).map(tokenize)

Map:   0%|          | 0/4168 [00:00<?, ? examples/s]

Map:   0%|          | 0/521 [00:00<?, ? examples/s]

In [ ]:
import warnings
warnings.filterwarnings('ignore')
# === Model ===
model = AutoModelForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=len(label_list))

# === Metrics ===
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids

    report = classification_report(labels, preds, output_dict=True, zero_division=0)

    return {
        "precision": precision_score(labels, preds, average="macro", zero_division=0),
        "recall": recall_score(labels, preds, average="macro", zero_division=0),
        "f1": f1_score(labels, preds, average="macro", zero_division=0),
        "f1_macro": report["macro avg"]["f1-score"],
        "precision_macro": report["macro avg"]["precision"],
        "recall_macro": report["macro avg"]["recall"],
    }

# === TrainingArguments với early stopping ===
args = TrainingArguments(
    output_dir="/content/drive/MyDrive/DS200_project/Vietnamese_Political_Events_Extraction/xlmr_event_type_classification",
    eval_strategy="steps",
    eval_steps=100,
    save_steps=100,
    logging_steps=100,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.01,
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
)

# === Trainer ===
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

# === Huấn luyện ===
trainer.train()

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss,Precision,Recall,F1,F1 Macro,Precision Macro,Recall Macro
100,2.384200,1.931102,0.243315,0.220262,0.182868,0.182868,0.243315,0.220262
200,1.664300,1.213708,0.400518,0.443692,0.403394,0.403394,0.400518,0.443692
300,1.220300,1.028144,0.499079,0.482963,0.453417,0.453417,0.499079,0.482963
400,1.041600,0.853443,0.602199,0.613443,0.592173,0.592173,0.602199,0.613443
500,0.905300,0.813044,0.718182,0.638736,0.627474,0.627474,0.718182,0.638736
600,0.764400,0.738643,0.750325,0.686677,0.694709,0.694709,0.750325,0.686677
700,0.651800,0.732110,0.776798,0.741409,0.739957,0.739957,0.776798,0.741409
800,0.657000,0.763118,0.780573,0.714266,0.726789,0.726789,0.780573,0.714266
900,0.702900,0.655575,0.781882,0.771125,0.769221,0.769221,0.781882,0.771125
1000,0.565000,0.679364,0.775015,0.759963,0.758044,0.758044,0.775015,0.759963


TrainOutput(global_step=1400, training_loss=0.889453729901995, metrics={'train_runtime': 383.9188, 'train_samples_per_second': 108.565, 'train_steps_per_second': 13.571, 'total_flos': 1473580661145600.0, 'train_loss': 0.889453729901995, 'epoch': 2.6871401151631478})

In [ ]:
# === Chuẩn bị dữ liệu test ===
test_data = convert_data(test_df, label_to_id)
test_dataset = Dataset.from_list(test_data).map(tokenize)

# === Dự đoán trên tập test ===
test_results = trainer.predict(test_dataset)
test_preds = np.argmax(test_results.predictions, axis=1)
test_labels = test_results.label_ids

# === Tính metrics ===
report = classification_report(test_labels, test_preds, target_names=label_list, digits=4, zero_division=0)
precision_macro = precision_score(test_labels, test_preds, average="macro", zero_division=0)
recall_macro = recall_score(test_labels, test_preds, average="macro", zero_division=0)
f1_macro = f1_score(test_labels, test_preds, average="macro", zero_division=0)

# === In kết quả ===
print("\n========== ĐÁNH GIÁ TRÊN TẬP TEST ==========")
print(f"Precision (macro): {precision_macro:.4f}")
print(f"Recall (macro):    {recall_macro:.4f}")
print(f"F1-score (macro):  {f1_macro:.4f}")

Map:   0%|          | 0/522 [00:00<?, ? examples/s]


========== ĐÁNH GIÁ TRÊN TẬP TEST ==========
Precision (macro): 0.8294
Recall (macro):    0.7911
F1-score (macro):  0.8011


In [ ]:
import torch

def predict_event_type(sentence: str):
    # Tokenize đầu vào
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding="max_length", max_length=256)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    # Dự đoán
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_class_id = torch.argmax(logits, dim=1).item()

    predicted_label = id_to_label[predicted_class_id]
    return predicted_label

# === Ví dụ sử dụng ===
input_sentence = "Phó Thủ tướng Nguyễn Chí Dũng đã tham dự cuộc họp về đầu tư công"
predicted_event_type = predict_event_type(input_sentence)

print(f"▶️ Câu nói: {input_sentence}")
print(f"🔍 Dự đoán event_type: {predicted_event_type}")


▶️ Câu nói: Phó Thủ tướng Nguyễn Chí Dũng đã tham dự cuộc họp về đầu tư công
🔍 Dự đoán event_type: Meeting


# ***Hướng joint learning***

## ***OneIE***

In [ ]:
import pandas as pd
import ast
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import XLMRobertaTokenizerFast, XLMRobertaModel
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import numpy as np
from tqdm import tqdm
import warnings
import os
import re

warnings.filterwarnings('ignore')

# === Dataset ===
class EventDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=128):
        # Ensure 'tokens', 'bio_tags', and 'argument_tags' are lists
        dataframe["tokens"] = dataframe["tokens"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
        dataframe["bio_tags"] = dataframe["bio_tags"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
        dataframe["argument_tags"] = dataframe["argument_tags"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

        # Filter out rows where lengths are inconsistent after evaluation
        dataframe = dataframe[
            (dataframe["tokens"].str.len() == dataframe["bio_tags"].str.len()) &
            (dataframe["tokens"].str.len() == dataframe["argument_tags"].str.len())
        ].reset_index(drop=True)


        self.data = dataframe.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.max_length = max_length

        all_trigger_labels = sum(self.data['bio_tags'].tolist(), [])
        all_arg_labels = sum(self.data['argument_tags'].tolist(), [])
        self.event_labels = sorted(self.data["event_type"].dropna().unique())


        self.trigger_labels = sorted(set(all_trigger_labels))
        self.arg_labels = sorted(set(all_arg_labels))


        self.trigger_encoder = LabelEncoder().fit(self.trigger_labels)
        self.arg_encoder = LabelEncoder().fit(self.arg_labels)
        self.event_encoder = LabelEncoder().fit(self.event_labels)

    def __len__(self): return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        tokens, trigger_tags, arg_tags = row['tokens'], row['bio_tags'], row['argument_tags']
        min_len = min(len(tokens), len(trigger_tags), len(arg_tags))
        tokens, trigger_tags, arg_tags = tokens[:min_len], trigger_tags[:min_len], arg_tags[:min_len]


        encoding = self.tokenizer(tokens, is_split_into_words=True, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt')
        word_ids = encoding.word_ids()

        o_trigger = self.trigger_encoder.transform(['O'])[0] if 'O' in self.trigger_encoder.classes_ else -100
        o_arg = self.arg_encoder.transform(['O'])[0] if 'O' in self.arg_encoder.classes_ else -100

        trigger_labels, arg_labels = [], []

        for word_idx in word_ids:
            if word_idx is None:
                trigger_labels.append(-100) # Use -100 for special tokens
                arg_labels.append(-100)     # Use -100 for special tokens
            elif word_idx < len(trigger_tags):
                 # Ensure the label exists in the encoder classes
                trigger_label = trigger_tags[word_idx]
                arg_label = arg_tags[word_idx]
                trigger_labels.append(self.trigger_encoder.transform([trigger_label])[0] if trigger_label in self.trigger_encoder.classes_ else -100)
                arg_labels.append(self.arg_encoder.transform([arg_label])[0] if arg_label in self.arg_encoder.classes_ else -100)
            else:
                trigger_labels.append(-100) # Handle potential index out of bounds after truncation
                arg_labels.append(-100)


        event_label = self.event_encoder.transform([row['event_type']])[0]


        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'trigger_labels': torch.tensor(trigger_labels, dtype=torch.long),
            'arg_labels': torch.tensor(arg_labels, dtype=torch.long),
            'event_label': torch.tensor(event_label, dtype=torch.long)
        }

In [ ]:
# === Mô hình ===
class JointModel(nn.Module):
    def __init__(self, model_name, num_trigger_labels, num_arg_labels, num_event_labels, dropout_rate=0.1):
        super().__init__()
        self.xlm_roberta = XLMRobertaModel.from_pretrained(model_name)
        hidden_size = self.xlm_roberta.config.hidden_size

        self.trigger_classifier = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_size, hidden_size // 2),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_size // 2, num_trigger_labels))

        self.arg_classifier = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_size, hidden_size // 2),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_size // 2, num_arg_labels))

        self.event_classifier = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_size, hidden_size // 2),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_size // 2, num_event_labels))

    def forward(self, input_ids, attention_mask):
        outputs = self.xlm_roberta(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = outputs.last_hidden_state
        cls_output = sequence_output[:, 0, :]

        return self.trigger_classifier(sequence_output), self.arg_classifier(sequence_output), self.event_classifier(cls_output)

In [ ]:
# === Training và Evaluation ===
def calculate_class_weights(labels, num_classes):
    # Filter out ignore_index (-100)
    valid_labels = [label for label in labels if label != -100]
    unique, counts = np.unique(valid_labels, return_counts=True)

    # Ensure all classes are represented, even if count is 0
    label_counts = dict(zip(unique, counts))
    weights = np.ones(num_classes)
    for i in range(num_classes):
        if i in label_counts:
            weights[i] = 1.0 / label_counts[i]
        else:
            # Handle classes not present in the batch, assign a small weight or average
            weights[i] = 1.0 / 1 # Assign a small count of 1 to avoid division by zero


    weights = weights / weights.sum() * num_classes
    return torch.FloatTensor(weights)

def train_model(model, train_loader, val_loader, num_epochs=80):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    all_trigger_labels = []
    all_arg_labels = []
    all_event_labels = []
    for batch in train_loader:
        # Only collect valid labels (not -100)
        all_trigger_labels.extend(batch['trigger_labels'].view(-1).tolist())
        all_arg_labels.extend(batch['arg_labels'].view(-1).tolist())
        all_event_labels.extend(batch['event_label'].tolist())


    trigger_weights = calculate_class_weights(all_trigger_labels, len(train_loader.dataset.trigger_labels)).to(device)
    arg_weights = calculate_class_weights(all_arg_labels, len(train_loader.dataset.arg_labels)).to(device)
    event_weights = calculate_class_weights(all_event_labels, len(train_loader.dataset.event_labels)).to(device)


    optimizer = optim.AdamW(model.parameters(), lr=2e-5)
    trigger_criterion = nn.CrossEntropyLoss(weight=trigger_weights, ignore_index=-100)
    arg_criterion = nn.CrossEntropyLoss(weight=arg_weights, ignore_index=-100)
    event_criterion = nn.CrossEntropyLoss(weight=event_weights)


    best_f1, patience = 0.0, 0
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for batch in tqdm(train_loader, desc=f'Epoch {epoch+1}'):
            optimizer.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            trigger_labels = batch['trigger_labels'].to(device)
            arg_labels = batch['arg_labels'].to(device)
            event_labels = batch['event_label'].to(device)

            trigger_logits, arg_logits, event_logits = model(input_ids, attention_mask)

            active_loss = attention_mask.view(-1) == 1
            trigger_loss = trigger_criterion(trigger_logits.view(-1, trigger_logits.size(-1))[active_loss], trigger_labels.view(-1)[active_loss])
            arg_loss = arg_criterion(arg_logits.view(-1, arg_logits.size(-1))[active_loss], arg_labels.view(-1)[active_loss])
            event_loss = event_criterion(event_logits, event_labels)

            loss = 0.4 * trigger_loss + 0.3 * arg_loss + 0.3 * event_loss
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            total_loss += loss.item()

        val_f1, _, _, _ = evaluate_model(model, val_loader, device)
        print(f"Epoch {epoch+1} - Val F1: {val_f1:.4f} - Loss: {total_loss:.4f}")

        if val_f1 > best_f1:
            best_f1 = val_f1
            torch.save(model.state_dict(), "/content/drive/MyDrive/DS200_project/Vietnamese_Political_Events_Extraction/xlmr_oneie/best_model.pt")
            torch.save(model.state_dict(), "/content/drive/MyDrive/DS200_project/Vietnamese_Political_Events_Extraction/streaming/best_model.pt")
            patience = 0
        else:
            patience += 1
            if patience >= 5:
                print("Early stopping.")
                break

    model.load_state_dict(torch.load("/content/drive/MyDrive/DS200_project/Vietnamese_Political_Events_Extraction/xlmr_oneie/best_model.pt"))
    return model

def evaluate_model(model, data_loader, device):
    model.eval()
    all_trigger_preds, all_trigger_labels = [], []
    all_arg_preds, all_arg_labels = [], []
    all_event_preds, all_event_labels = [], []

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            trigger_labels = batch['trigger_labels'].cpu().numpy()
            arg_labels = batch['arg_labels'].cpu().numpy()
            event_labels = batch['event_label'].cpu().numpy()

            trigger_logits, arg_logits, event_logits = model(input_ids, attention_mask)
            trigger_preds = torch.argmax(trigger_logits, dim=-1).cpu().numpy()
            arg_preds = torch.argmax(arg_logits, dim=-1).cpu().numpy()
            event_preds = torch.argmax(event_logits, dim=-1).cpu().numpy()

            for i in range(len(trigger_labels)):
                mask = attention_mask[i].cpu().numpy().astype(bool)
                all_trigger_preds.extend(trigger_preds[i][mask])
                all_trigger_labels.extend(trigger_labels[i][mask])
                all_arg_preds.extend(arg_preds[i][mask])
                all_arg_labels.extend(arg_labels[i][mask])
                all_event_preds.append(event_preds[i])
                all_event_labels.append(event_labels[i])

    trigger_report = classification_report(all_trigger_labels, all_trigger_preds, output_dict=True, zero_division=0)
    arg_report = classification_report(all_arg_labels, all_arg_preds, output_dict=True, zero_division=0)
    event_report = classification_report(all_event_labels, all_event_preds, output_dict=True, zero_division=0)

    avg_f1 = np.mean([
        trigger_report['macro avg']['f1-score'],
        arg_report['macro avg']['f1-score'],
        event_report['macro avg']['f1-score']
    ])
    return avg_f1, trigger_report, arg_report, event_report

In [ ]:
# --- Load dữ liệu ---
df = pd.read_csv("results.csv")
df["tokens"] = df["tokens"].apply(ast.literal_eval)
df = df[df['event_type'].notna()].reset_index(drop=True)

train_size = int(0.8 * len(df))
train_df, val_df = df[:train_size], df[train_size:]

tokenizer = XLMRobertaTokenizerFast.from_pretrained('xlm-roberta-base')
train_dataset = EventDataset(train_df, tokenizer)
val_dataset = EventDataset(val_df, tokenizer)

val_dataset.trigger_encoder = train_dataset.trigger_encoder
val_dataset.arg_encoder = train_dataset.arg_encoder
val_dataset.event_encoder = train_dataset.event_encoder
val_dataset.trigger_labels = train_dataset.trigger_labels
val_dataset.arg_labels = train_dataset.arg_labels
val_dataset.event_labels = train_dataset.event_labels

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)

model = JointModel(
    model_name='xlm-roberta-base',
    num_trigger_labels=len(train_dataset.trigger_labels),
    num_arg_labels=len(train_dataset.arg_labels),
    num_event_labels=len(train_dataset.event_labels)
)

trained_model = train_model(model, train_loader, val_loader)
final_f1, trigger_report, arg_report, event_report = evaluate_model(trained_model, val_loader, torch.device('cuda' if torch.cuda.is_available() else 'cpu'))

print(f"\n✅ Final Macro F1: {final_f1:.4f}")

print("\n========== ĐÁNH GIÁ TRÊN TẬP TEST ==========")
print("----- Trigger BIO -----")
print(f"Precision (macro): {trigger_report['macro avg']['precision']:.4f}")
print(f"Recall (macro):    {trigger_report['macro avg']['recall']:.4f}")
print(f"F1-score (macro):  {trigger_report['macro avg']['f1-score']:.4f}")

print("\n----- Argument -----")
print(f"Precision (macro): {arg_report['macro avg']['precision']:.4f}")
print(f"Recall (macro):    {arg_report['macro avg']['recall']:.4f}")
print(f"F1-score (macro):  {arg_report['macro avg']['f1-score']:.4f}")

print("\n----- Event Type -----")
print(f"Precision (macro): {event_report['macro avg']['precision']:.4f}")
print(f"Recall (macro):    {event_report['macro avg']['recall']:.4f}")
print(f"F1-score (macro):  {event_report['macro avg']['f1-score']:.4f}")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

...

Epoch 43 - Val F1: 0.6005 - Loss: 12.2326


Epoch 44: 100%|██████████| 582/582 [02:39<00:00,  3.64it/s]


Epoch 44 - Val F1: 0.6029 - Loss: 12.0793


Epoch 45: 100%|██████████| 582/582 [02:39<00:00,  3.64it/s]


Epoch 45 - Val F1: 0.5992 - Loss: 16.7236
Early stopping.

✅ Final Macro F1: 0.6121

========== ĐÁNH GIÁ TRÊN TẬP TEST ==========
----- Trigger BIO -----
Precision (macro): 0.3868
Recall (macro):    0.4991
F1-score (macro):  0.4165

----- Argument -----
Precision (macro): 0.6396
Recall (macro):    0.6693
F1-score (macro):  0.6491

----- Event Type -----
Precision (macro): 0.8140
Recall (macro):    0.7496
F1-score (macro):  0.7707


In [ ]:
print(f"Trigger labels: {len(train_dataset.trigger_labels)}")
print(f"Argument labels: {len(train_dataset.arg_labels)}")
print(f"Event labels: {len(train_dataset.event_labels)}")

Trigger labels: 26
Argument labels: 7
Event labels: 14


In [ ]:
model = JointModel(
    model_name='xlm-roberta-base',
    num_trigger_labels=len(train_dataset.trigger_labels),
    num_arg_labels=len(train_dataset.arg_labels),
    num_event_labels=len(train_dataset.event_labels)
)

# Load mô hình đã lưu
model.load_state_dict(torch.load("/content/drive/MyDrive/DS200_project/Vietnamese_Political_Events_Extraction/xlmr_oneie/best_model.pt"))
model.eval()

JointModel(
  (xlm_roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
     

In [ ]:
import pickle
import os

# === Lưu các encoder ra file .pkl ===
encoder_dir = "/content/drive/MyDrive/DS200_project/Vietnamese_Political_Events_Extraction/oneie_encoders"
os.makedirs(encoder_dir, exist_ok=True)

with open(os.path.join(encoder_dir, "trigger_encoder.pkl"), "wb") as f:
    pickle.dump(train_dataset.trigger_encoder, f)

with open(os.path.join(encoder_dir, "arg_encoder.pkl"), "wb") as f:
    pickle.dump(train_dataset.arg_encoder, f)

with open(os.path.join(encoder_dir, "event_encoder.pkl"), "wb") as f:
    pickle.dump(train_dataset.event_encoder, f)

print("Đã lưu xong các file .pkl")

# === Đọc lại các file .pkl và in ra độ dài các classes ===
with open(os.path.join(encoder_dir, "trigger_encoder.pkl"), "rb") as f:
    trigger_encoder_loaded = pickle.load(f)

with open(os.path.join(encoder_dir, "arg_encoder.pkl"), "rb") as f:
    arg_encoder_loaded = pickle.load(f)

with open(os.path.join(encoder_dir, "event_encoder.pkl"), "rb") as f:
    event_encoder_loaded = pickle.load(f)

print("Số lượng trigger labels:", len(trigger_encoder_loaded.classes_))
print("Số lượng argument labels:", len(arg_encoder_loaded.classes_))
print("Số lượng event labels:", len(event_encoder_loaded.classes_))

Đã lưu xong các file .pkl
Số lượng trigger labels: 26
Số lượng argument labels: 7
Số lượng event labels: 14


In [ ]:
import torch
import pickle
import os
from transformers import XLMRobertaTokenizerFast

# === Dự đoán một câu ===

def predict_single_instance(model, tokenizer, sentence_tokens, dataset_encoders, max_length=128):
    model.eval()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    encoding = tokenizer(sentence_tokens, is_split_into_words=True, return_tensors='pt', padding='max_length', truncation=True, max_length=max_length)
    word_ids = encoding.word_ids()
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        trigger_logits, arg_logits, event_logits = model(input_ids, attention_mask)
        trigger_preds = torch.argmax(trigger_logits, dim=-1).cpu().numpy()[0]
        arg_preds = torch.argmax(arg_logits, dim=-1).cpu().numpy()[0]
        event_pred = torch.argmax(event_logits, dim=-1).cpu().item()

    trigger_labels, arg_labels = [], []
    for i, word_idx in enumerate(word_ids):
        if word_idx is not None and word_idx < len(sentence_tokens):
            # Sử dụng inverse_transform từ đối tượng encoder đã tải
            trigger_labels.append(dataset_encoders['trigger'].inverse_transform([trigger_preds[i]])[0])
            arg_labels.append(dataset_encoders['arg'].inverse_transform([arg_preds[i]])[0])
        # Xử lý các token padding hoặc special tokens nếu cần, ví dụ gán nhãn 'O'
        elif word_idx is None:
             # Thêm nhãn 'O' cho các token đặc biệt (CLS, SEP, PAD)
             trigger_labels.append('O')
             arg_labels.append('O')
        # else: # word_idx >= len(sentence_tokens) - có thể xảy ra với truncation=False
        #     trigger_labels.append('O')
        #     arg_labels.append('O')


    # Điều chỉnh để độ dài của predicted_tags khớp với độ dài của tokens gốc
    # Loại bỏ các nhãn 'O' được thêm vào cho padding nếu số lượng nhãn dự đoán nhiều hơn tokens gốc
    if len(trigger_labels) > len(sentence_tokens):
        trigger_labels = [label for i, label in enumerate(trigger_labels) if word_ids[i] is not None][:len(sentence_tokens)]
        arg_labels = [label for i, label in enumerate(arg_labels) if word_ids[i] is not None][:len(sentence_tokens)]
    elif len(trigger_labels) < len(sentence_tokens):
        # Trường hợp bị truncation, cần xử lý cẩn thận hơn hoặc thông báo cho người dùng
        print(f"Warning: Sentence truncated. Predicted tags length ({len(trigger_labels)}) is less than original tokens length ({len(sentence_tokens)})")


    event_label = dataset_encoders['event'].inverse_transform([event_pred])[0]

    return {
        'tokens': sentence_tokens,
        'trigger_tags': trigger_labels,
        'argument_tags': arg_labels,
        'event_type': event_label
    }

# Đường dẫn đến thư mục chứa encoder
encoder_dir = "/content/drive/MyDrive/DS200_project/Vietnamese_Political_Events_Extraction/oneie_encoders"

# Tải encoder
with open(os.path.join(encoder_dir, "trigger_encoder.pkl"), "rb") as f:
    trigger_encoder = pickle.load(f)
with open(os.path.join(encoder_dir, "arg_encoder.pkl"), "rb") as f:
    arg_encoder = pickle.load(f)
with open(os.path.join(encoder_dir, "event_encoder.pkl"), "rb") as f:
    event_encoder = pickle.load(f)


# --- Inference ví dụ ---
raw_sentence = "Tiếp tục chương trình Kỳ họp thứ 9, chiều 20/6, Quốc hội họp phiên toàn thể tại Hội trường, nghe báo cáo về dự thảo Nghị quyết của Quốc hội về cơ chế xử lý khó khăn, vướng mắc do quy định của pháp luật."
sentence_tokens = raw_sentence.strip().split()  # tách câu thành từng từ đơn giản
tokenizer = XLMRobertaTokenizerFast.from_pretrained('xlm-roberta-base')

result = predict_single_instance(
    model,  # nhớ dùng model đã load từ best_model.pt
    tokenizer,
    sentence_tokens,
    {
        'trigger': trigger_encoder,
        'arg': arg_encoder,
        'event': event_encoder
    }
)

print("\n===== PREDICTION =====")
# Đảm bảo số lượng token và nhãn khớp nhau khi in ra
num_tokens_to_print = min(len(result['tokens']), len(result['trigger_tags']), len(result['argument_tags']))
for i in range(num_tokens_to_print):
    print(f"{result['tokens'][i]:<10} Trigger: {result['trigger_tags'][i]:<15} Argument: {result['argument_tags'][i]}")
print(f"\n➡️ Event Type: {result['event_type']}")


===== PREDICTION =====
Tiếp       Trigger: O               Argument: O
tục        Trigger: O               Argument: O
chương     Trigger: O               Argument: O
trình      Trigger: O               Argument: O
Kỳ         Trigger: O               Argument: O
họp        Trigger: B-Meeting       Argument: O
thứ        Trigger: O               Argument: O
9,         Trigger: O               Argument: O
chiều      Trigger: O               Argument: O
20/6,      Trigger: O               Argument: B-Arg-Time
Quốc       Trigger: O               Argument: I-Arg-Time
hội        Trigger: O               Argument: I-Arg-Time
họp        Trigger: O               Argument: O
phiên      Trigger: O               Argument: B-Arg-Subject
toàn       Trigger: O               Argument: B-Arg-Subject
thể        Trigger: B-Meeting       Argument: O
tại        Trigger: B-Meeting       Argument: O
Hội        Trigger: B-Meeting       Argument: O
trường,    Trigger: B-Meeting       Argument: O
nghe       Tr